<a href="https://colab.research.google.com/github/deybvagm/natural-language-processing/blob/master/fastext_imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de lenguaje natural con el conjunto de datos Imdb y la librería fasttext

[Imdb](https://www.imdb.com/) es una base de datos en línea que contiene información de peliculas, programas de televisión, entre otro contenido audiovisual relacionado. Para trabajar temas de procesamiento de lenguaje natural, se dispone de una **base de datos de 50.000 reviews** de peliculas proporcionados por usuarios de la plataforma.


[Fasttext](https://fasttext.cc/) es una librería de procesamiento de lenguaje natural creada por el laboratorio de inteligencia artificial de facebook. En este caso específico, será utilizada para clasificar los reviews de usuarios como positivos o negativos. Un comentario **negativo** en este contexto, es el que tiene de **0-4 estrellas** en su calificación. Un comentario es **positivo** si en la review el usuario dio de **7-10 estrellas**.

### Instalación de librería para procesamiento de texto

In [0]:
! pip install fasttext

### Integración con Google Drive
Es de utilidad ya que el modelo esta almacenado en Drive y por lo tanto para hacer uso de él podemos hacer esta integración con el fin de acceder a los datos que allí se tengan

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Ahora nos vamos a la carpeta de Drive donde está almacenado el modelo

In [0]:
%cd drive/My\ Drive/nlp

/content/drive/My Drive/nlp


In [0]:
! ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [0]:
! ls aclImdb/train/neg -1 | wc -l

ls: cannot open directory 'aclImdb/train/neg': Input/output error
0


### Import de librerías a usar y definición de funciones auxiliares para pre-procesamiento de texto

La función auxiliar es encargada de convertir el texto a minúsculas y además de dejar el texto en un formato que pueda ser leído por la librería `fasttext`.  Por ejemplo un texto como **`El restaurante esmuy bueno!`** es  convertido en **`__label__5 el restaurante es muy bueno!`**

In [0]:
import fasttext
import re

def strip_formatting(string):
    string = string.lower()
    string = re.sub(r"([.!?,'/()])", r" \1 ", string)
    return string

### Entrenamiento del modelo de Machine Learning

Para entrenar el modelo de machine learning se requiere la información de las reviews de clientes junto con su calificación (número de estrellas). Además de esto es necesario dividir los datos en dos conjuntos, uno de entrenamiento con el cual se realiza el aprendizaje, y el otro de pruebas usado para hacer la evaluación del modelo obtenido. Una vez realizado el entrenamiento, se guarda el modelo en una ruta indicada (en este caso una ruta en Google Drive)

In [0]:
! ls aclImdb/

imdbEr.txt  imdb.vocab	README	test  train


In [0]:
training_path = Path("aclImdb") / "train"
class1 = "neg"
class2 = "pos"

files = training_path.glob("/*.txt")
print(training_path)
print(dir(training_path))
baba_path = training_path.joinpath("baba")
print(training_path)
print(baba_path)
baba_path.glob("*.txt")

# my_file = next(files)
# print(my_file)
# print(type(my_file))

aclImdb/train
['__bytes__', '__class__', '__delattr__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__fspath__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rtruediv__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__truediv__', '_accessor', '_cached_cparts', '_closed', '_cparts', '_drv', '_flavour', '_format_parsed_parts', '_from_parsed_parts', '_from_parts', '_hash', '_init', '_make_child', '_make_child_relpath', '_opener', '_parse_args', '_parts', '_pparts', '_raise_closed', '_raw_open', '_root', '_str', 'absolute', 'anchor', 'as_posix', 'as_uri', 'chmod', 'cwd', 'drive', 'exists', 'expanduser', 'glob', 'group', 'home', 'is_absolute', 'is_block_device', 'is_char_device', 'is_dir', 'is_fifo', 'is_file', 'is_reserved', 'is_socket', 'is_symlink', 'iterdir', 'joinpath', 'lchmod', 'lstat', 'm

In [0]:
! ls

 aclImdb		      imdb_lstm.py   xlnet_cased_L-12_H-768_A-12
 aclImdb_pred		      lstm_model    'xlnet commands.txt'
 exp			      predictions    yelp
 fasttest_imdb_testing.txt    proc_data      yelp_model
 fasttest_imdb_training.txt   xlnet


In [0]:
train_file = Path("fasttext_imdb_training.txt")
test_file = Path("fasttext_imdb_testing.txt")

training_path = Path("aclImdb") / "train"
test_path = Path("aclImdb") / "test"


In [0]:
from pathlib import Path


def extract_data(path_to_review, label):

    with path_to_review.open('r') as review_file:
        review_data = review_file.read()
    text = review_data.replace("\n", " ")
    text = strip_formatting(text)
    fasttext_line = "__label__{} {}".format(label, text)
    return fasttext_line + "\n"


def prepare_dataset(base_path, out_path):
    reviews = ''
    classes = {"pos": 1, "neg": 0}
    i = 0
    for cl, v in classes.items():
        for review_filename in base_path.joinpath(cl).glob("*.txt"):
            text = extract_data(review_filename, v)
            reviews = reviews + text
            i += 1
            if i % 1000 == 0:
                print('review: {}'.format(i))
    with out_path.open('w+') as out:
        out.write(reviews)   
  

prepare_dataset(training_path, train_file)
prepare_dataset(test_path, test_file)


In [0]:
import random

with train_file.open('r') as f:
  lines = f.readlines()

random.shuffle(lines)

shuffled_training_data = 'fasttext_imdb_training_shuffled.txt'

with open(shuffled_training_data, 'w') as f:
    for line in lines:
        f.write("%s\n" % line)


In [0]:
model = fasttext.train_supervised(shuffled_training_data, word_ngrams=2)


In [0]:
model_name = 'fasttext_imdb_model.bin'
model.save_model(model_name)

### Predicciones sobre el modelo

Con el modelo creado y almacenado en disco, ahora es posible realizar predicciones sobre nuevos reviews para identificar automáticamente el número de estrellas que el modelo asignaría a los reviews. Es importante que para los nuevos reviews se haga un pre-procesamiento de los datos para dejarlos en el formato que exige la librería

In [0]:
# Reviews to check
reviews = [
    "This restaurant literally changed my life. This is the best food I've ever eaten!",
    "I hate this place so much. They were mean to me.",
    "I don't know. It was ok, I guess. Not really sure what to say."
]

# Pre-process the text of each review so it matches the training format
preprocessed_reviews = list(map(strip_formatting, reviews))

# Load the model
classifier = fasttext.load_model(model_name)

# Get fastText to classify each review with the model
labels, probabilities = classifier.predict(preprocessed_reviews, 1)

# Print the results
for review, label, probability in zip(reviews, labels, probabilities):
    pred = int(label[0][-1])

    print("{} review ({}% confidence)".format('positive' if pred == 1 else 'negative', int(probability[0] * 100)))
    print(review)
    print()

positive review (99% confidence)
This restaurant literally changed my life. This is the best food I've ever eaten!

negative review (99% confidence)
I hate this place so much. They were mean to me.

negative review (93% confidence)
I don't know. It was ok, I guess. Not really sure what to say.



### Descarga del modelo

La siguiente instrucción realiza la descarga del modelo

In [0]:

try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download(model_name) 

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 47736, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

### ¿Qué tan bueno es el modelo para realizar predicciones?

El siguiente paso es determinar la capacidad del modelo para hacer predicciones sobre reviews que no ha visto. Para esto se usa el conjunto de datos de prueba

In [0]:
eval = classifier.test(test_file.name)
print(eval)

(25000, 0.84664, 0.84664)


El resultado nos dice que se evaluaron 25000 reviews, de las cuales el modelo dice que el 84% de ellas fueron exactamente bien clasificadas como positivas o negativas. 